## 7.3.4 워터 셰드
- floodfill과 비슷한 방식으로, 연속된 영역을 찾으며 seed를 하나가 아닌 여러 곳을 사용하며 marker라고 함
- markers = cv2.watershed(img, markers)
    - markers : 입력 영상과 크기가 같은 1차원 배열 (int32)

In [14]:
import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')
rows, cols = img.shape[:2]
img_draw = img.copy()

# 마커 생성, 모든 요소는 0으로 초기화
marker = np.zeros((rows, cols), np.int32)
markerId = 1
colors = []
isDragging = False

# mouse event func.
def onMouse(event, x, y, flags, param):
    global img_draw, marker, markerId, isDragging
    if event == cv2.EVENT_LBUTTONDOWN:
        isDragging = True
        # 각 마커의 아이디와 현위치의 색상 값을 쌍으로 매핑해서 저장
        colors.append((markerId, img[y, x]))

    elif event == cv2.EVENT_MOUSEMOVE:
        if isDragging:
            # 마우스 좌표에 해당하는 마커의 좌표에 동일한 마커 아이디로 채우기
            marker[y, x] = markerId
            # 마커를 표시한 곳을 빨간색 점으로 표시
            cv2.circle(img_draw, (x, y), 3, (0, 0, 255), -1)
            cv2.imshow('watershed', img_draw)

    elif event == cv2.EVENT_LBUTTONUP:
        if isDragging:
            isDragging = False
            markerId += 1
    
    elif event == cv2.EVENT_RBUTTONDOWN:
        # 모은 마커를 이용해 워터셰드 적용
        cv2.watershed(img, marker)
        # 마커에 -1로 표시된 경계를 green line
        img_draw[marker == -1] = (0, 255, 0)
        for mid, color in colors:
            # 같은 마커 아이디값 갖는 영역을 마커를 선택한 색상으로 채우기
            img_draw[marker == mid] = color
        cv2.imshow('watershed', img_draw)

# result
cv2.imshow('watershed', img)
cv2.setMouseCallback('watershed', onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 7.3.5 그랩컷(grabcut)
- mask, bgdModel, fgdModel = cv2.grabCut(img, mask, rect, bgdModel, fgdModel, iterCount[, mode])
    - mask : 입력 영상과 크기가 같은 1채널로 배열, 배경과 전경을 구분하는 값 저장
        - cv2.GC_BGD : 확실한 배경 (0)
        - cv2.GC_FGD : 확실한 전경 (1)
        - cv2.GC_PR_BGD : 아마도 배경 (3)
        - cv2.GC_PR_FGD : 아마도 전경 (4)
    - rect : 전경이 있을 것으로 추측되는 영역의 사각형 좌표, tuple(x1, y1, x2, y2)
    - bgdModel, fgdModel : 함수 내에서 사용할 임시 배열 버퍼 (재사용할 경우 수정 X)
    - iterCount : 반복 횟수
    - mode : 동작 방법
        - cv2.GC_INIT_WITH_RECT 에 지정한 좌표를 기준으로 그랩컷 수행
        - cv2.GC_INIT_WITH_MASK 에 지정한 값을 기준으로 그랩컷 수행
        - cv2.GC_EVAL : 재시도

In [16]:
import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')
img_draw = img.copy()
mask = np.zeros(img.shape[:2], dtype=np.uint8)
rect = [0, 0, 0, 0] # rect 초기화
mode = cv2.GC_EVAL # 그랩컷 초기 모드

# 배경 및 전경 모델 버퍼
bgdmodel = np.zeros((1, 65), dtype=np.float64)
fgdmodel = np.zeros((1, 65), dtype=np.float64)

# mouse event func.
def onMouse(event, x, y, flags, param):
    global mouse_mode, rect, mask, mode
    if event == cv2.EVENT_LBUTTONDOWN:
        if flags <= 1: # 아무키도 안누른 경우
            mode = cv2.GC_INIT_WITH_RECT # drag 시작, 사각형 모드
            rect[:2] = x, y # 시작 좌표 저장
        
    elif event == cv2.EVENT_MOUSEMOVE and flags and cv2.EVENT_FLAG_LBUTTON:
        if mode == cv2.GC_INIT_WITH_RECT: # drag 진행중
            img_temp = img.copy()
            # drag 사각형 화면에 표시
            cv2.rectangle(img_temp, (rect[0], rect[1]), (x, y), (0, 255, 0), 2)
            cv2.imshow('img', img_temp)

    elif flags > 1 : # 키가 눌러진 상태
        mode = cv2.GC_INIT_WITH_MASK # 마스크 모드
        if flags and cv2.EVENT_FLAG_CTRLKEY: # ctrl 분명한 전경
            cv2.circle(img_draw, (x, y), 3, (255, 255, 255), -1)
            # 마스크에 GC_FGD로 채우기
            cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)

        if flags and cv2.EVENT_FLAG_SHIFTKEY: # shift 분명한 배경
            cv2.circle(img_draw, (x, y), 3, (0, 0, 0), -1)
            # 마스크에 GC_BGD로 채우기
            cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
    
    elif event == cv2.EVENT_LBUTTONUP:
        if mode == cv2.GC_INIT_WITH_RECT: # drag 진행중
            rect[2:]  = x, y # 사각형 마지막 좌표 수집
            # 사각형 화면에 표시
            cv2.rectangle(img_draw, (rect[0], rect[1]), (x, y), (255, 0, 0), 2)
            cv2.imshow('img', img_draw)
        # 그랩컷 적용
        cv2.grabCut(img, mask, tuple(rect), bgdmodel, fgdmodel, 1, mode)
        img2 = img.copy()
        # 마스크에 확실한 배경, 아마도 배경으로 표시된 영역을 0dmfh codnrl
        img2[(mask==cv2.GC_BGD) | (mask==cv2.GC_PR_BGD)] = 0
        cv2.imshow('grabCut', img2)
        mode = cv2.GC_EVAL # 그랩컷 모드 리셋

# result
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)
while True:
    if cv2.waitKey(0) & 0xFF == 27:
        break
cv2.destroyAllWindows()

### 7.3.6 평균 이동 필터

- dst = cv2.pyrMeanShiftFiltering(src, sp, sr[, dst, maxLevel, termcrit])
    - sp : 공간 윈도 반지름 크기
    - sr : 색상 윈도 반지름 크기
    - maxLevel : 이미지 피라미드 최대 레벨
    - termcrit : 반복 중지 요건
        - type = cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS : 중지 형식
            - cv2.TERM_CRITERIA_EPS : 정확도가 최소 정확도 보다 작아지면 중지
            - cv2.TERM_CRITERIA_MAX_ITER : 최대 반복 횟수(max_iter)에 도달하면 중지
            - cv2.TERM_CRITERIA_COUNT : cv2.TERM_CRITERIA_MAX_ITER와 동일
        - max_iter = 5 (최대 반복 횟수)
        - epsilon = 1 (최소 정확도)

In [17]:
import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')

# trackbar event func.
def onChange(x):
    # sp, sr, level 선택값 수집
    sp = cv2.getTrackbarPos('sp', 'img')
    sr = cv2.getTrackbarPos('sr', 'img')
    lv = cv2.getTrackbarPos('lv', 'img')

    # 평균 이동 필터 적용
    mean = cv2.pyrMeanShiftFiltering(img, sp, sr, None, lv)
    # 변환 이미지 출력
    cv2.imshow('img', np.hstack((img, mean)))

# 초기 화면 출력
cv2.imshow('img', np.hstack((img, img)))
# 트랙바
cv2.createTrackbar('sp', 'img', 0, 100, onChange)
cv2.createTrackbar('sr', 'img', 0, 100, onChange)
cv2.createTrackbar('lv', 'img', 0, 5, onChange)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 7.4 실전 워크숍
### 7.4.1 도형 알아맞히기

In [20]:
import cv2
import numpy as np

img1 = cv2.imread('img/5shapes.jpg')
img2 = img1.copy()

# to gray scale
gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# 스레시홀드 바이너리 이미지로 만들어 검은 배경에 흰 전경으로 반전
ret, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# find contour
contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    # 각 컨투어에 근사 컨투어로 단순화
    epsilon = 0.01*cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # 꼭지점의 개수
    vertices = len(approx)
    print('vertices: ', vertices)

    # center pt
    mmt = cv2.moments(contour)
    cx, cy = int(mmt['m10']/mmt['m00']), int(mmt['m01']/mmt['m00'])

    name = 'Unknown'
    if vertices == 3: # 꼭지점 3 = triangle
        name = 'Triangle'
        color = (0, 255, 0)
    elif vertices == 4:
        x, y, w, h = cv2.boundingRect(contour)
        if abs(w-h) <= 3: # 정사각형
            name = 'Square'
            color = (0, 125, 255)

        else: 
            name = 'Rectangle'
            color = (0, 0, 255)

    elif vertices == 10: 
        name = 'Star'
        color = (255, 255, 0)

    elif vertices >= 15 : # 꼭지점 3 = triangle
        name = 'Circle'
        color = (0, 255, 255)


    # 컨투어 선 그리기
    cv2.drawContours(img2, [contour], -1, color, -1)
    cv2.putText(img2, name, (cx-50, cy), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (100, 100, 100), 1)

# result
cv2.imshow('input shapes', img1)
cv2.imshow('recognizing shapes', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

vertices:  10
vertices:  16
vertices:  3
vertices:  4
vertices:  4


### 7.4.2 문서 스캐너

In [3]:
import cv2
import numpy as np

img = cv2.imread('img/paper.jpg')
cv2.imshow('original', img)
cv2.waitKey(0)
win_name = 'scan'
draw = img.copy()

# to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # noise remove
edged = cv2.Canny(gray, 75, 200)
cv2.imshow(win_name, edged)
cv2.waitKey(0)

# find contour
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# draw all contour
cv2.drawContours(draw, contours, -1, (0, 255, 0))
cv2.imshow(win_name, edged)
cv2.waitKey(0)

# 컨투어들 중 영역 크기순으로 정렬
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
for contour in contours:
    # 영역이 큰 컨투어부터 근사 컨투어로 단순화
    peri = cv2.arcLength(contour, True) # 둘레
    vertices = cv2.approxPolyDP(contour, 0.02*peri, True)
    if len(vertices) == 4: # 근사한 꼭지점이 4개면 중지
        break

pts = vertices.reshape(4, 2)  # Nx1x2 -> 4x2
for x, y in pts:
    cv2.circle(draw, (x, y), 10, (0, 255, 0), -1)
cv2.imshow(win_name, draw)
cv2.waitKey(0)
merged = np.hstack((img, draw))

# 좌표 상하좌우 찾기
sm = pts.sum(axis=1)
diff = np.diff(pts, axis=1)

topLeft = pts[np.argmin(sm)]
bottomRight = pts[np.argmax(sm)]
topRight = pts[np.argmin(diff)]
bottomLeft = pts[np.argmax(diff)]

# 변환 전 4개 좌표
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])

# 변환 후 사용할 서류의 폭과 높이 계산
w1 = abs(bottomRight[0] - bottomLeft[0])
w2 = abs(topRight[0] - topLeft[0])
h1 = abs(topRight[1] - bottomRight[1])
h2 = abs(topLeft[1] - bottomLeft[1])
width = max([w1, w2])
height = max([h1, h2])

# 변환 후 4개의 좌표
pts2 = np.float32([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]])
mtrx = cv2.getPerspectiveTransform(pts1, pts2)

# result
result = cv2.warpPerspective(img, mtrx, (width, height))
cv2.imshow(win_name, result)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 7.4.3 동전 개수 세기

In [4]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread('img/coins_connected.jpg')
rows, cols = img.shape[:2]
cv2.imshow('original', img)


# 동전 표면을 흐릿하게 피라미드평균시프트 적용
mean = cv2.pyrMeanShiftFiltering(img, 20, 50)
cv2.imshow('mean', mean)
# 바이너리 이미지 변환
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3,3), 0)

_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)
# 거리 변환
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 3)
# 거리 값을 0 ~255로 변환
dst = ( dst / (dst.max() - dst.min()) * 255 ).astype(np.uint8)
cv2.imshow('dst', dst)

# 거리 변환결과에서 로칼 최대 값 구하기
## 팽창 적용(동전 크기 정도의 구조화 요소 필요),
localMx = cv2.dilate(dst, np.ones((50,50), np.uint8))
## 로칼 최대 값 저장 할 배열 생성
lm = np.zeros((rows, cols), np.uint8)
## 팽창 적용전 이미지와 같은 픽셀이 로컬 최대 값이므로 255로 설정
lm[(localMx==dst) & (dst != 0)] = 255
cv2.imshow('localMx', lm)

# 로컬 최대값으로 색 채우기
## 로컬 최대 값이 있는 좌표 구하기
seeds = np.where(lm ==255)
seed = np.stack( (seeds[1], seeds[0]), axis=-1)
## 색 채우기를 위한 채우기 마스크 생성
fill_mask = np.zeros((rows+2, cols+2), np.uint8)
for x,y in seed:
    ## 로칼 최대값을 시드로해서 평균 시프트 영상에 색채우기 
    ret = cv2.floodFill(mean, fill_mask, (x,y), (255,255,255), \
                                            (10,10,10), (10,10,10))
cv2.imshow('floodFill', mean)

# 색 채우기 적용한 영상에 다시 거리 변환 적용
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0)

ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
dst = ( (dst / (dst.max() - dst.min())) * 255 ).astype(np.uint8)
cv2.imshow('dst2', dst)

# 거리 변환 결과값의 절반 이상을 차지한 영역은 확실한 전경으로 설정
ret, sure_fg = cv2.threshold(dst, 0.5*dst.max(), 255,0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과를 반전해서 확실한 배경 찾기
_, bg_th = cv2.threshold(dst, 0.3*dst.max(),  255, cv2.THRESH_BINARY_INV)
bg_dst = cv2.distanceTransform(bg_th, cv2.DIST_L2, 5)
bg_dst = ( (bg_dst / (bg_dst.max() - bg_dst.min())) * 255 ).astype(np.uint8)
ret, sure_bg = cv2.threshold(bg_dst, 0.3*bg_dst.max(), 255,cv2.THRESH_BINARY)
cv2.imshow('sure_bg', sure_bg)


# 불확실한 영역 설정 : 확실한 배경을 반전해서 확실한 전경을 빼기
ret, inv_sure_bg = cv2.threshold(sure_bg, 127, 255,cv2.THRESH_BINARY_INV)
unkown = cv2.subtract(inv_sure_bg, sure_fg)
cv2.imshow('unkown', unkown)

# 연결된 요소 레이블링
_, markers = cv2.connectedComponents(sure_fg)

# 레이블링을 1씩 증가 시키고 0번 레이블 알 수 없는 영역을 0번 레이블로 설정
markers = markers+1
markers[unkown ==255] = 0
print("워터쉐드 전:", np.unique(markers))
colors = []
marker_show = np.zeros_like(img)
for mid in np.unique(markers): # 선택한 마커 아이디 갯수 만큼 반복
    color = [int(j) for j in np.random.randint(0,255, 3)]
    colors.append((mid, color))
    marker_show[markers==mid] = color
    coords = np.where(markers==mid)
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
cv2.imshow('before', marker_show)

# 레이블링이 완성된 마커로 워터 쉐드 적용
markers = cv2.watershed(img, markers)
print("워터쉐드 후:", np.unique(markers))

for mid, color in colors: # 선택한 마커 아이디 갯수 만큼 반복
    marker_show[markers==mid] = color
    coords = np.where(markers==mid)
    if coords[0].size <= 0 : 
        continue
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
marker_show[markers==-1] = (0,255,0)
cv2.imshow('watershed marker', marker_show)

img[markers==-1] = (0,255,0)
cv2.imshow('watershed', img)

# 동전 추출을 위한 마스킹 생성
mask = np.zeros((rows, cols), np.uint8)
# 배경 마스크 생성
mask[markers!=1] = 255
# 배경 지우기
nobg = cv2.bitwise_and(img, img, mask=mask)
# 동전만 있는 라벨 생성 (배경(1), 경계(-1) 없는)
coin_label = [l for l in np.unique(markers) if (l != 1 and l !=-1)]
# 동전 라벨 순회 하면서 동전 영역만 추출
for i, label in enumerate(coin_label):
    mask[:,:] = 0
    # 해당 동전 추출 마스크 생성
    mask[markers ==label] = 255
    # 동전 영역만 마스크로 추출
    coins = cv2.bitwise_and(img, img, mask=mask)
    # 동전 하나만 있는 곳에서 최외곽 컨투어 추출
    contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # 동전을 감싸는 사각형 좌표
    x,y,w,h = cv2.boundingRect(contour[0])
    # 동전 영역만 추출해서 출력
    coin = coins[y:y+h, x:x+w]
    cv2.imshow('coin%d'%(i+1), coin)
    cv2.imwrite('img/coin_test/coin%d.jpg'%(i+1), coin)
cv2.waitKey()
cv2.destroyAllWindows()

워터쉐드 전: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
워터쉐드 후: [-1  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
